In [1]:
import re
from datetime import datetime, timedelta

import pandas as pd
from bs4 import BeautifulSoup
import requests

class KBOResultBot:    
    def __init__(self, date, team='롯데'):
        self.date = self.convert_to_yyyy_mm_dd(date)
        self.team = team
        
    def convert_to_yyyy_mm_dd(self, date_str):
        if date_str == '오늘':
            return datetime.now().strftime("%Y-%m-%d")
        
        formats = ["%Y년 %m월 %d일", "%Y/%m/%d", "%Y-%m-%d", "%Y %m %d", "%Y.%m.%d", "%Y.%m.%d",
                   "%y년 %m월 %d일", "%y/%m/%d", "%y-%m-%d", "%y %m %d", "%y.%m.%d", "%y.%m.%d", "%y%m%d",
                  ]

        for fmt in formats:
            try:
                date_obj = datetime.strptime(date_str, fmt)
                return date_obj.strftime("%Y-%m-%d")
            except ValueError:
                pass

        return "날짜 형식을 인식할 수 없습니다."
        
    def summary(self):
        if self.date == "날짜 형식을 인식할 수 없습니다.":
            return ["날짜 형식을 인식할 수 없습니다."]
        
        url = f'https://statiz.sporki.com/schedule/?m=daily&date={self.date}'
        r = requests.get(url).text
        soup = BeautifulSoup(r, 'lxml')

        boxes = soup.find_all("div", "box_cont")
        summaries = []
        
        for i, box in enumerate(boxes):
            if self.team in box.text:
                box_header = soup.find_all("div", "box_head")[i]
                if '경기취소' in box_header.text:
                    summaries.append(['경기취소'])
                    continue
                result_table = self.table(box)
                result_text = self.result_text(box, result_table)
                    
                summaries.append(f'<b>{self.date}</b>\n{result_text}\n<pre>{result_table.to_markdown(tablefmt="grid")}</pre>')
                    
        return summaries
                
    def table(self, box):
        columns = [b.text for b in box.find_all('th')]
        
        team1 = []
        team2 = []
        
        for td in box.find_all('tr')[1].find_all('td'):
            for p_tag in td.find_all('p'):
                p_tag.decompose()  # <p> 태그 제거
            text = td.get_text(strip=True)  # 텍스트 부분 추출
            team1.append(text)


        for td in box.find_all('tr')[2].find_all('td'):
            for p_tag in td.find_all('p'):
                p_tag.decompose()  # <p> 태그 제거
            text = td.get_text(strip=True)  # 텍스트 부분 추출
            team2.append(text)
        
        result_table = pd.DataFrame([team1, team2], columns=columns).set_index('팀').T
        
        return result_table
    
    def result_text(self, soup, result_table):
        if result_table.iloc[-1, 0] == result_table.iloc[-1, 1]:
            result = "무승부"
        else:
            team1 = result_table.columns[0]
            team2 = result_table.columns[1]
            
            team1_score = result_table.loc['R', team1]
            team2_score = result_table.loc['R', team2]
            
            if team1_score > team2_score:
                result = f"(승) {team1} {team1_score} : {team2_score} {team2} (패)"
            else:
                result = f"(패) {team1} {team1_score} : {team2_score} {team2} (승)"
            
            try:
                winner = soup.find_all("ul", class_="win")[0].find_all(['span', 'a', 'em'])[:3]
                result += f'\n{winner[0].text}: {winner[1].text} {winner[2].text}'
                
                loser = soup.find_all("ul", class_="lose")[0].find_all(['span', 'a', 'em'])[:3]
                result += f'\n{loser[0].text}: {loser[1].text} {loser[2].text}'
                
                saver = soup.find_all("ul", class_="save")[0].find_all(['span', 'a', 'em'])[:3]
                result += f'\n{saver[0].text}: {saver[1].text} {saver[2].text}'
                
            except:
                0
        return result
    
    
    def best_worst(self):
        if self.date == "날짜 형식을 인식할 수 없습니다.":
            return ["날짜 형식을 인식할 수 없습니다."]
        
        url = f'https://statiz.sporki.com/schedule/?m=daily&date={self.date}'
        r = requests.get(url).text
        soup = BeautifulSoup(r, 'lxml')

        boxes = soup.find_all("div", "box_cont")
        links = []
        
        for i, box in enumerate(boxes):
            if self.team in box.text:
                box_header = soup.find_all("div", "box_head")[i]
                if '경기취소' in box_header.text:
                    summaries.append(['경기취소'])
                    continue
                game_summary_button = box.find('a', text='경기요약')
                if game_summary_button:
                    link = game_summary_button['href']
                    links.append(f'https://statiz.sporki.com{link}')
        bests_worsts = []
        for link in links:
            best_worst = f'<b>{self.date}</b>'
            titles = ['경기 Best 5', '경기 Worst 5']
            if link != '경기취소':
                r = requests.get(link).text
                soup = BeautifulSoup(r, 'lxml')

                tables = soup.find_all("table")[1:3]
                
                for i, table in enumerate(tables):
                    columns = [col.text.strip() for col in table.find_all('th')]
                    
                    rows = table.find_all('tr')
                    data = []
                    for row in rows[1:]:
                        cols = row.find_all('td')
                        row_data = [col.text.strip() for col in cols]
                        data.append(row_data)
                    best_worst += f"\n{titles[i]}\n<pre>{pd.DataFrame(data, columns=columns)[['이름', '활약']].to_markdown(tablefmt='grid', index=None)}</pre>"
            
            else:
                best_worst += '경기취소'
                    
            bests_worsts.append(best_worst)
        
        return bests_worsts
    
    
    def score_scene(self):
        if self.date == "날짜 형식을 인식할 수 없습니다.":
            return ["날짜 형식을 인식할 수 없습니다."]
        
        url = f'https://statiz.sporki.com/schedule/?m=daily&date={self.date}'
        r = requests.get(url).text
        soup = BeautifulSoup(r, 'lxml')

        boxes = soup.find_all("div", "box_cont")
        links = []
        
        for i, box in enumerate(boxes):
            if self.team in box.text:
                box_header = soup.find_all("div", "box_head")[i]
                if '경기취소' in box_header.text:
                    summaries.append(['경기취소'])
                    continue
                game_summary_button = box.find('a', text='경기요약')
                if game_summary_button:
                    link = game_summary_button['href']
                    links.append(f'https://statiz.sporki.com{link}')
        
        scenes = []
        for link in links:
            scene = f'<b>{self.date}</b>'
            if link != '경기취소':
                r = requests.get(link).text
                soup = BeautifulSoup(r, 'lxml')

                table = soup.find_all("table")[-1]
                columns = [col.text.strip() for col in table.find_all('th')]

                rows = table.find_all('tr')
                data = []
                for row in rows[1:]:
                    cols = row.find_all('td')
                    row_data = [col.text.strip() for col in cols]
                    data.append(row_data)

                scene += f"\n<pre>{pd.DataFrame(data, columns=columns)[['이닝', '투수', '타자', '결과']].to_markdown(tablefmt='grid', index=None)}</pre>"
                scene += f"\n<pre>{pd.DataFrame(data, columns=columns)[['이전상황', '이후상황']].to_markdown(tablefmt='grid', index=None)}</pre>"
            
            else:
                scene += '경기취소'
                    
            scenes.append(scene)
        
        return scenes

In [5]:
print(*KBOResultBot('23 9 9').summary())

<b>2023-09-09</b>
(승) 롯데 5 : 2 NC (패)
승: 박세웅 (6승 7패)
패: 송명기 (3승 8패)
세: 김원중 (5승 4패 26세)
<pre>+----+--------+------+
|    |   롯데 |   NC |
+====+========+======+
| 1  |      0 |    1 |
+----+--------+------+
| 2  |      0 |    0 |
+----+--------+------+
| 3  |      0 |    1 |
+----+--------+------+
| 4  |      0 |    0 |
+----+--------+------+
| 5  |      0 |    0 |
+----+--------+------+
| 6  |      4 |    0 |
+----+--------+------+
| 7  |      1 |    0 |
+----+--------+------+
| 8  |      0 |    0 |
+----+--------+------+
| 9  |      0 |    0 |
+----+--------+------+
| R  |      5 |    2 |
+----+--------+------+</pre> <b>2023-09-09</b>
(패) 롯데 5 : 6 NC (승)
승: 김영규 (1승 4패)
패: 진승현 (1승 2패)
세: 이용찬 (4승 3패 22세)
<pre>+----+--------+------+
|    |   롯데 | NC   |
+====+========+======+
| 1  |      2 | 0    |
+----+--------+------+
| 2  |      0 | 1    |
+----+--------+------+
| 3  |      2 | 2    |
+----+--------+------+
| 4  |      0 | 0    |
+----+--------+------+
| 5  |      0 | 0    |
+----+---

In [2]:
print(*KBOResultBot('23 9 9').score_scene())

<b>2023-09-09</b>
<pre>+--------+--------+--------+-------------------+
| 이닝   | 투수   | 타자   | 결과              |
+========+========+========+===================+
| 1회말  | 박세웅 | 권희동 | 중견수 방면 1루타 |
+--------+--------+--------+-------------------+
| 3회말  | 박세웅 | 마틴   | 우익수 방면 1루타 |
+--------+--------+--------+-------------------+
| 6회초  | 송명기 | 전준우 | 좌익수 방면 1루타 |
+--------+--------+--------+-------------------+
| 6회초  | 김영규 | 정훈   | 좌익수 뒤 홈런    |
+--------+--------+--------+-------------------+
| 7회초  | 김시훈 | 안치홍 | 중견수 방면 1루타 |
+--------+--------+--------+-------------------+</pre>
<pre>+---------------+---------------+
| 이전상황      | 이후상황      |
+===============+===============+
| 2사 1,2루 0:0 | 2사 1,3루 0:1 |
+---------------+---------------+
| 2사 3루 0:1   | 2사 1루 0:2   |
+---------------+---------------+
| 1사 2,3루 0:2 | 1사 2루 2:2   |
+---------------+---------------+
| 1사 2루 2:2   | 1사  4:2      |
+---------------+---------------+
| 2사 3루 4:2   | 2사 1루 5:2   |
+---------------+-----------

In [3]:
print(*KBOResultBot('23 9 9').best_worst())

<b>2023-09-09</b>
경기 Best 5
<pre>+--------+---------------------------------------+
| 이름   | 활약                                  |
+========+=======================================+
| 정훈   | 2타수 1안타 1홈런 2타점 0삼진 1볼넷   |
+--------+---------------------------------------+
| 전준우 | 4타수 2안타 0홈런 2타점 0삼진 0볼넷   |
+--------+---------------------------------------+
| 안치홍 | 5타수 2안타 0홈런 1타점 1삼진 0볼넷   |
+--------+---------------------------------------+
| 마틴   | 3타수 2안타 0홈런 1타점 1삼진 1볼넷   |
+--------+---------------------------------------+
| 박세웅 | 6.2이닝 2실점 2자책 6안타 9삼진 1볼넷 |
+--------+---------------------------------------+</pre>
경기 Worst 5
<pre>+--------+---------------------------------------+
| 이름   | 활약                                  |
+========+=======================================+
| 김영규 | 0.2이닝 1실점 1자책 2안타 0삼진 0볼넷 |
+--------+---------------------------------------+
| 윤동희 | 4타수 0안타 0홈런 0타점 1삼진 0볼넷   |
+--------+---------------------------------------+
| 오영수 | 3타수 0안타 0홈런 0타점 2삼진 0볼넷   |


In [4]:
KBOResultBot('23 5 6').summary()

[['경기취소']]

In [6]:
print(*KBOResultBot('84 5 6').summary())

<b>1984-05-06</b>
(패) 롯데 1 : 2 OB (승)
승: 계형철 (3승 1패)
패: 임호균 (3승 3패)
<pre>+----+--------+------+
|    |   롯데 | OB   |
+====+========+======+
| 1  |      1 | 0    |
+----+--------+------+
| 2  |      0 | 0    |
+----+--------+------+
| 3  |      0 | 0    |
+----+--------+------+
| 4  |      0 | 0    |
+----+--------+------+
| 5  |      0 | 0    |
+----+--------+------+
| 6  |      0 | 0    |
+----+--------+------+
| 7  |      0 | 0    |
+----+--------+------+
| 8  |      0 | 2    |
+----+--------+------+
| 9  |      0 | -    |
+----+--------+------+
| R  |      1 | 2    |
+----+--------+------+</pre>


In [7]:
class KBOYearRecordBot:
    
    def __init__(self, year):
        self.year = self.convert_to_yyyy(year)
        
        
    def convert_to_yyyy(self, year):
        year = int(year)
        if year < 100:
            if year >= 82:
                year += 1900
            else:
                year += 2000
        
        elif year >= int(datetime.today().year):
            year = "날짜 형식을 인식할 수 없습니다."
        
        return year
    
    
    def year_category(self, year):
        if year < 1990:
            return 1980
        elif year < 2000:
            return 1990
        elif year < 2010:
            return 2000
        elif year < 2020:
            return 2010
        else:
            return 2020
    
    def team_records(self):
        y = self.year_category(self.year)
        if y == "날짜 형식을 인식할 수 없습니다.":
            return [y]
        
        url = f"https://www.koreabaseball.com/Record/History/Team/Record.aspx?startYear={y}&halfSc=T"
        r = requests.get(url).text
        soup = BeautifulSoup(r, 'lxml')
        if self.year >= 1990:
            table_idx = self.year % 10
        else:
            table_idx = self.year % 10 - 2
        trs = soup.find_all('table', 'tData')[table_idx].find_all('tr')
        
        records = [[]]
        names = []
        column_name = [th.text for th in trs[0].find_all('th')]
        
        for row in trs[1:-1]:
            columns = row.find_all('td')
            if len(columns) > 0:
                team_record = [
                    row.find('th').text.strip(),
                    int(columns[0].text),
                    int(columns[1].text),
                    int(columns[2].text),
                    int(columns[3].text),
                    float(columns[4].text),
                    float(columns[5].text),
                    float(columns[6].text)
                ]

                records[-1].append(team_record)
            else:
                records.append([])
                names.append(row.find('th').text)

        if len(records) == 1:
            df = pd.DataFrame(records[0], columns=column_name, index=range(1, len(records[0])+1))
            df.index.name = '순위'
            return [f"<b>{self.year}년도 정규리그 순위</b>\n<pre>{df.reset_index().loc[:, :'무'].to_markdown(index=None, tablefmt='grid')}</pre>",
                    f"<pre>{df.reset_index().loc[:, [str(self.year), '타율', '평균자책점', '승률']].to_markdown(index=None, tablefmt='grid')}</pre>"]
        else:
            df1 = pd.DataFrame(records[1], columns=column_name, index=range(1, len(records[1])+1))
            df2 = pd.DataFrame(records[2], columns=column_name, index=range(1, len(records[2])+1))
            df1.index.name = names[0].replace('<', '').replace('>', '').strip()
            df2.index.name = names[1].replace('<', '').replace('>', '').strip()
        
            return [f"<b>{self.year}년도 정규리그 순위</b>\n<pre>{df1.loc[:, :'무'].to_markdown(tablefmt='grid')}</pre>", 
                    f"<pre>{df1.loc[:, [str(self.year), '타율', '평균자책점', '승률']].to_markdown(index=None, tablefmt='grid')}</pre>",
                    f"<pre>{df2.loc[:, :'무'].to_markdown(tablefmt='grid')}</pre>",
                   f"<pre>{df2.loc[:, [str(self.year), '타율', '평균자책점', '승률']].to_markdown(index=None, tablefmt='grid')}</pre>"]

In [8]:
print(*KBOYearRecordBot(98).team_records(), sep='\n')

<b>1998년도 정규리그 순위</b>
<pre>+--------+-----------------+--------+------+------+------+
|   순위 | 1998            |   경기 |   승 |   패 |   무 |
+========+=================+========+======+======+======+
|      1 | 현대 유니콘스   |    126 |   81 |   45 |    0 |
+--------+-----------------+--------+------+------+------+
|      2 | LG 트윈스       |    126 |   63 |   62 |    1 |
+--------+-----------------+--------+------+------+------+
|      3 | 삼성 라이온즈   |    126 |   66 |   58 |    2 |
+--------+-----------------+--------+------+------+------+
|      4 | OB 베어스       |    126 |   61 |   62 |    3 |
+--------+-----------------+--------+------+------+------+
|      5 | 해태 타이거즈   |    126 |   61 |   64 |    1 |
+--------+-----------------+--------+------+------+------+
|      6 | 쌍방울 레이더스 |    126 |   58 |   66 |    2 |
+--------+-----------------+--------+------+------+------+
|      7 | 한화 이글스     |    126 |   55 |   66 |    5 |
+--------+-----------------+--------+------+------+------+
|      8 | 롯데

In [9]:
print(*KBOYearRecordBot('99').team_records(), sep='\n')

<b>1999년도 정규리그 순위</b>
<pre>+------------+---------------+--------+------+------+------+
|   드림리그 | 1999          |   경기 |   승 |   패 |   무 |
+============+===============+========+======+======+======+
|          1 | 롯데 자이언츠 |    132 |   75 |   52 |    5 |
+------------+---------------+--------+------+------+------+
|          2 | 두산 베어스   |    132 |   76 |   51 |    5 |
+------------+---------------+--------+------+------+------+
|          3 | 현대 유니콘스 |    132 |   68 |   59 |    5 |
+------------+---------------+--------+------+------+------+
|          4 | 해태 타이거즈 |    132 |   60 |   69 |    3 |
+------------+---------------+--------+------+------+------+</pre>
<pre>+---------------+--------+--------------+--------+
| 1999          |   타율 |   평균자책점 |   승률 |
+===============+========+==============+========+
| 롯데 자이언츠 |  0.291 |         4.18 |  0.591 |
+---------------+--------+--------------+--------+
| 두산 베어스   |  0.284 |         4.58 |  0.598 |
+---------------+--------+-----------

In [10]:
print(*KBOYearRecordBot('84').team_records(), sep='\n')

<b>1984년도 정규리그 순위</b>
<pre>+--------+-------------------+--------+------+------+------+
|   순위 | 1984              |   경기 |   승 |   패 |   무 |
+========+===================+========+======+======+======+
|      1 | 롯데   자이언츠   |    100 |   50 |   48 |    2 |
+--------+-------------------+--------+------+------+------+
|      2 | 삼성   라이온즈   |    100 |   55 |   45 |    0 |
+--------+-------------------+--------+------+------+------+
|      3 | OB     베어스     |    100 |   58 |   41 |    1 |
+--------+-------------------+--------+------+------+------+
|      4 | MBC    청룡       |    100 |   51 |   48 |    1 |
+--------+-------------------+--------+------+------+------+
|      5 | 해태   타이거즈   |    100 |   43 |   54 |    3 |
+--------+-------------------+--------+------+------+------+
|      6 | 삼미   슈퍼스타즈 |    100 |   38 |   59 |    3 |
+--------+-------------------+--------+------+------+------+</pre>
<pre>+-------------------+--------+--------------+--------+
| 1984              |   타율 | 

In [11]:
print(*KBOYearRecordBot('92').team_records(), sep='\n')

<b>1992년도 정규리그 순위</b>
<pre>+--------+-----------------+--------+------+------+------+
|   순위 | 1992            |   경기 |   승 |   패 |   무 |
+========+=================+========+======+======+======+
|      1 | 롯데 자이언츠   |    126 |   71 |   55 |    0 |
+--------+-----------------+--------+------+------+------+
|      2 | 빙그레 이글스   |    126 |   81 |   43 |    2 |
+--------+-----------------+--------+------+------+------+
|      3 | 해태 타이거즈   |    126 |   71 |   54 |    1 |
+--------+-----------------+--------+------+------+------+
|      4 | 삼성 라이온즈   |    126 |   67 |   57 |    2 |
+--------+-----------------+--------+------+------+------+
|      5 | OB 베어스       |    126 |   56 |   66 |    4 |
+--------+-----------------+--------+------+------+------+
|      6 | 태평양 돌핀스   |    126 |   56 |   67 |    3 |
+--------+-----------------+--------+------+------+------+
|      7 | LG 트윈스       |    126 |   53 |   70 |    3 |
+--------+-----------------+--------+------+------+------+
|      8 | 쌍방